In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [71.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/gr

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



Step 1: Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

In [6]:
#Load in a sql function to use columns
from pyspark.sql.functions import col

vine_reviews_df = vine_df.filter(col("total_votes") >20)
vine_reviews_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|                N|
| RO6JNMUJDE1LP|          5|           37|         50|   N|                N|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|     

Step 2: Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

In [15]:
vine_percentage_df = vine_reviews_df.withColumn("Helpful_Vote_Percentage", vine_reviews_df.helpful_votes / vine_reviews_df.total_votes)
vine_percentage_limited_df = vine_percentage_df.filter(col("Helpful_Vote_Percentage") > .50)
vine_percentage_limited_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Vote_Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|     0.9124087591240876|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|     0.9333333333333333|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|     0.8846153846153846|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|     0.7619047619047619|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|     0.8947368421052632|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
only showing top 5 rows



Step 3: Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [17]:
vine_yes_df = vine_percentage_limited_df.filter(col("vine")=="Y")
vine_yes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Vote_Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N|      0.918918918918919|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|     0.8648648648648649|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|                   0.88|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|               0.953125|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|                    1.0|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
only showing top 5 rows



Step 4: Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [18]:
vine_no_df = vine_percentage_limited_df.filter(col("vine")=="N")
vine_no_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Vote_Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|     0.9124087591240876|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|     0.9333333333333333|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|     0.8846153846153846|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|     0.7619047619047619|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|     0.8947368421052632|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
only showing top 5 rows



Step 5: Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [24]:
#I chose to create a table based on the combined dataframe, using the groupby method

from pyspark.sql.functions import when,count,lit
ratings_total_df = vine_percentage_limited_df.groupBy("vine").agg(
    count(col("vine")).alias("Total Reviews"),
    count(when(col("star_rating") == 5, True)).alias("5 Star Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("% 5 Star Reviews")).show()

+----+-------------+--------------+------------------+
|vine|Total Reviews|5 Star Reviews|  % 5 Star Reviews|
+----+-------------+--------------+------------------+
|   Y|           56|            18|32.142857142857146|
|   N|        26299|         14616|55.576257652382225|
+----+-------------+--------------+------------------+



In [25]:
#checking total reviews for vine
vine_yes_df.count()

56

In [26]:
#checking total reviews for no vine
vine_no_df.count()

26299